In [1]:
import pandas as pd

train_df = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv').set_index('id').reset_index(drop=True)
test_df = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv').set_index('id').reset_index(drop=True)


In [2]:
train_df.shape

(90615, 9)

In [3]:
test_df.shape

(60411, 8)

In [4]:
train_df = pd.concat([train_df, pd.get_dummies(train_df['Sex'], dtype=int)], axis=1).drop(columns=['Sex'])






In [5]:
test_df = pd.concat([test_df, pd.get_dummies(test_df['Sex'], dtype=int)], axis=1).drop(columns=['Sex'])


In [6]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [7]:
import matplotlib.pyplot as plt

from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.decomposition import PCA

from collections import Counter

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from xgboost import XGBRegressor

from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings("ignore")


In [8]:
from sklearn.metrics import mean_squared_log_error

def rmsle_score(y_true, y_pred):
    msle = mean_squared_log_error(abs(y_true), abs(y_pred))
    return np.sqrt(msle)

rmsle_scorer = make_scorer(rmsle_score)

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

X = train_df.drop(columns=['Rings'])
y = train_df['Rings']

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import make_scorer


def rmsle_score(y_true, y_pred):
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2))




X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


models = [
    
    CatBoostRegressor(task_type='CPU', verbose=False, random_state=42)  
]


kf = KFold(n_splits=10, shuffle=True, random_state=42)
scores_list = []

for model in models:
    try:
        scores = cross_val_score(model, X_train, y_train, scoring=make_scorer(rmsle_score), cv=kf)
        scores_list.append(scores)
        print(f"Model: {model.__class__.__name__}")
        print("RMSLE scores mean:", np.mean(scores))
        print("\n")
    except Exception as e:
        print(f"Failed to train {model.__class__.__name__} due to {e}")




Model: CatBoostRegressor
RMSLE scores mean: 0.14945312666382982




In [10]:
feature_names = X_train.columns
for idx, model in enumerate(models[1:]):  
    model.fit(X_train, y_train)  
    importances = model.feature_importances_
    importance_data = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=importance_data.sort_values('Importance', ascending=False))
    plt.title(f'{model.__class__.__name__} Feature Importance')
    plt.tight_layout()
    plt.show()


In [11]:
sub = pd.read_csv('/kaggle/input/playground-series-s4e4/sample_submission.csv')

cat = CatBoostRegressor(
    task_type='GPU', 
    verbose=False, 
    random_state=42,
    iterations = 4885, 
    learning_rate = 0.0010440584402121896, 
    bagging_temperature = 0.3583926555184656,
    depth = 5, 
    l2_leaf_reg = 0.59441367755379071,
    random_strength = 0.0096918283504264173,
    border_count = 252,
)



from catboost import CatBoostRegressor


cat = CatBoostRegressor(task_type='CPU', verbose=False, random_state=42)


try:
    cat.fit(X, y)
    print("CatBoost model trained successfully.")
except Exception as e:
    print("Failed to train CatBoost model:", e)


if 'cat' in locals():  
    try:
        sub['Rings'] = cat.predict(test_df)
        print("Predictions added to 'sub' DataFrame successfully.")
    except Exception as e:
        print("Failed to make predictions:", e)


CatBoost model trained successfully.
Predictions added to 'sub' DataFrame successfully.


In [12]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
cross_val_score(cat, X, y, scoring=make_scorer(rmsle_score), cv=kf).mean()

0.14936783253610555

In [13]:
sub['Rings'] = abs(sub['Rings'])

In [14]:
sub.to_csv('submission.csv', index=False)